Libraries

In [7]:
import json
import pandas as pd
import numpy as np
from textblob import TextBlob

Load data

In [17]:
#Documents
#tip_path = "/Users/Andy/Desktop/yelp_academic_dataset_tip.json"
tip_path="../../yelp_dataset/yelp_academic_dataset_tip.json"
tip_doc=open(tip_path,encoding="utf8").readlines()

business_path="../../yelp_dataset/yelp_academic_dataset_business.json"
business_doc=open(business_path,encoding="utf8").readlines()



#Dataframes
busList = []
for bus in business_doc:
    busList.append(json.loads(bus))

dfBus = pd.DataFrame.from_records(busList)

tipList = []
for tip in tip_doc:
    tipList.append(json.loads(tip))

df_Bus = pd.DataFrame.from_records(busList)
df_Tip = pd.DataFrame.from_records(tipList)

df_comb = pd.merge(df_Bus, df_Tip, on='business_id', how='outer')
df_comb=df_comb[df_comb['text'].notna()] # filter on empty tips

## Task 1

#### Task 1a

In [18]:
#TextBlob(df_comb.iloc[2]["text"]).sentiment
#map TextBlob(df_comb.iloc[i]["text"]).sentiment and append as column

df_comb['sentiment'] = df_comb[['text']].applymap(lambda tip: TextBlob(tip).sentiment.polarity)

In [19]:
df_comb.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,text,date,compliment_count,sentiment
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...",CZ517SU7uQDc3Aa17SN6Bw,Their prices are enormously inflated. I was ch...,2013-02-17 22:29:53,0.0,0.05500
1,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...",M0DM77NUD5_w4EHuH2vZNg,Go to Eagle Guns or Point Blank Range instead,2013-09-26 21:43:54,0.0,0.00000
2,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...",46HhzhpBfTdTSB5ceTx_Og,Friendly staff!,2012-05-28 18:57:21,0.0,0.46875


#### Task 1b

In [47]:
df_sentiment_by_state = df_comb.groupby("state",as_index=False)['sentiment'].mean()


In [56]:
print("Mean Sentiment by State")
print("")

#filter out candadian provinces
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

df_sentiment_by_USstate = df_sentiment_by_state[df_sentiment_by_state['state'].isin(states)]
print(df_sentiment_by_USstate)

Mean Sentiment by State

   state  sentiment
1     AL   0.177778
2     AR   0.400205
3     AZ   0.315210
5     CA   0.176626
6     CO   0.092188
7     CT   0.302083
8     HI   0.107143
9     IL   0.297415
11    MI   0.400000
12    MO   0.450000
13    NC   0.304907
14    NE   0.282143
15    NV   0.282558
16    NY   0.430124
17    OH   0.306991
19    PA   0.282545
21    SC   0.321160
22    TX   0.322010
23    VA   0.200000
24    VT  -0.450000
25    WA   0.062731
26    WI   0.287927


#### Task 1c

In [58]:
import plotly
import plotly.graph_objects as go
#plotly.offline.plot(fig, filename='map.html')


In [60]:
fig=go.Figure(data=go.Choropleth(
    locations=df_sentiment_by_USstate["state"],
    z=df_sentiment_by_USstate['sentiment'].astype(float),
    locationmode='USA-states',
    colorscale='Reds',
    colorbar_title="Sentiment",
)
)

fig.update_layout(
    title_text='Mean Yelp tip Seniment Analisis',
    geo_scope='usa'
)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Task 2

#### Task 2a

In [12]:
state_map = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California", "CO": "Colorado",
    "CT": "Connecticut", "DE": "Delaware", "DC": "Washington, D.C.", "FL": "Florida", "GA": "Georgia",
    "HI": "Hawaii", "ID": "Idaho", "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas",
    "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland", "MA": "Massachusetts",
    "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi", "MO": "Missouri", "MT": "Montana",
    "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York",
    "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma", "OR": "Oregon",
    "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota",
    "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont", "VA": "Virginia", "WA": "Washington",
    "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming"
}


#### Task 2b

#### Task 2c